# Finetune GAP-text2SQL for SQL Query Generation on Bank Account Fraud Dataset Suite

[![Open in SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/vxcent/sagemaker_gap_demo/blob/main/rat-sql-gap/notebook.ipynb) 

## Step 0. Install all necessary packages

### Open up a Terminal window through Sagemaker Studio Lab for issuing the following commands

In [ ]:
conda activate default
conda install -y python=3.7
conda install -y -c conda-forge jsonnet openjdk
conda install -y pytorch=1.5 cudatoolkit=10.2 -c pytorch

### Run the following cells to install Python dependencies

In [1]:
%pip install -r requirements.txt

  Using cached transformers-3.0.0-py3-none-any.whl (754 kB)
  Using cached jsonnet-0.14.0-cp37-cp37m-linux_x86_64.whl
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached networkx-2.6.3-py3-none-any.whl (1.9 MB)
  Using cached entmax-1.1-py3-none-any.whl (12 kB)
  Using cached pyrsistent-0.14.11-cp37-cp37m-linux_x86_64.whl
  Using cached bpemb-0.2.12-py3-none-any.whl (19 kB)
  Using cached stanford_corenlp-3.9.2-py2.py3-none-any.whl (11 kB)
  Using cached torchtext-0.3.1-py3-none-any.whl (62 kB)
  Using cached Flask-2.2.5-py3-none-any.whl (101 kB)
  Using cached attrs-23.1.0-py3-none-any.whl (61 kB)
  Using cached asdl-0.1.5-py3-none-any.whl
  Using cached protobuf-3.20.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.0 MB)
  Using cached astor-0.8.1-py2.py3-none-any.whl (27 kB)
  Using cached sacremoses-0.0.53-py3-none-any.whl
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached filelock-3.12

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 887.5 MB 5.7 kB/s s eta 0:00:01
  Using cached gensim-4.2.0-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (24.1 MB)
  Using cached corenlp_protobuf-3.8.0-py2.py3-none-any.whl (15 kB)
  Using cached Werkzeug-2.2.3-py3-none-any.whl (233 kB)
  Using cached itsdangerous-2.1.2-py3-none-any.whl (15 kB)
  Using cached charset_normalizer-3.2.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (175 kB)
  Using cached certifi-2023.7.22-py3-none-any.whl (158 kB)
  Using cached urllib3-2.0.4-py3-none-any.whl (123 kB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Using cached scipy-1.7.3-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_6

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/studio-lab-
[nltk_data]     user/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/studio-lab-
[nltk_data]     user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
mkdir third_party
wget http://nlp.stanford.edu/software/stanford-corenlp-full-2018-10-05.zip
unzip stanford-corenlp-full-2018-10-05.zip -d third_party/


### Download the Pretrained Model and Finetune Checkpoint (in terminal shell)

#### Download Finetune Checkpoint

In [3]:
cd sagemaker_gap_demo/rat-sql-gap

[Errno 2] No such file or directory: 'sagemaker_gap_demo/rat-sql-gap'
/home/studio-lab-user/sagemaker_gap_demo/rat-sql-gap


In [ ]:
mkdir -p logdir/bart_run_1/bs\=12\,lr\=1.0e-04\,bert_lr\=1.0e-05\,end_lr\=0e0\,att\=1/
curl https://gap-text2sql-public.s3.amazonaws.com/checkpoint-artifacts/gap-finetuned-checkpoint -o logdir/bart_run_1/bs\=12\,lr\=1.0e-04\,bert_lr\=1.0e-05\,end_lr\=0e0\,att\=1/model_checkpoint-00041000

#### Download the Pretrained Model

In [17]:
mkdir -p pretrained_checkpoint
curl https://gap-text2sql-public.s3.amazonaws.com/checkpoint-artifacts/pretrained-checkpoint -o pretrained_checkpoint/pytorch_model.bin

SyntaxError: invalid syntax (3773467495.py, line 1)

In [4]:
import json
import os
import _jsonnet
import os

In [5]:
from seq2struct.commands.infer import Inferer
from seq2struct.datasets.spider import SpiderItem
from seq2struct.utils import registry

In [6]:
import torch

#### The gap-run.jsonnet file includes metadata used for experiment tracking
#### The most important field `model_config_args` includes hyperperameters used for inferences

#### To learn more about the usage of the 'facebook/bart-large' model, checkout the [Huggin Face 🤗 Documentation](https://huggingface.co/facebook/bart-large) 

In [8]:
exp_config = json.loads(
    _jsonnet.evaluate_file(
        "experiments/spider-configs/gap-run.jsonnet"))


In [9]:
model_config_path = exp_config["model_config"]
model_config_args = exp_config.get("model_config_args")

In [10]:
infer_config = json.loads(
    _jsonnet.evaluate_file(
        model_config_path, 
        tla_codes={'args': json.dumps(model_config_args)}))

In [11]:
infer_config["model"]["encoder_preproc"]["db_path"] = "data/sqlite_files/"

In [12]:
inferer = Inferer(infer_config)

WARNING <class 'seq2struct.models.enc_dec.EncDecModel.Preproc'>: superfluous {'name': 'EncDec'}


In [14]:
model_dir = exp_config["logdir"] + "/bs=12,lr=1.0e-04,bert_lr=1.0e-05,end_lr=0e0,att=1"
checkpoint_step = exp_config["eval_steps"][0]

In [15]:
print(model_dir)
print(checkpoint_step)

logdir/bart_run_1/bs=12,lr=1.0e-04,bert_lr=1.0e-05,end_lr=0e0,att=1
41000


In [18]:
model = inferer.load_model(model_dir, checkpoint_step)

WARNING <class 'seq2struct.models.enc_dec.EncDecModel'>: superfluous {'decoder_preproc': {'grammar': {'clause_order': None, 'end_with_from': True, 'factorize_sketch': 2, 'include_literals': False, 'infer_from_conditions': True, 'name': 'spider', 'output_from': True, 'use_table_pointer': True}, 'save_path': 'data/spider-bart/nl2code-1115,output_from=true,fs=2,emb=bart,cvlink', 'use_seq_elem_rules': True}, 'encoder_preproc': {'bart_version': 'facebook/bart-large', 'compute_cv_link': True, 'compute_sc_link': True, 'db_path': 'data/sqlite_files/', 'fix_issue_16_primary_keys': True, 'include_table_name_in_column': False, 'save_path': 'data/spider-bart/nl2code-1115,output_from=true,fs=2,emb=bart,cvlink'}}


Parameter containing:
tensor([[-0.0370,  0.1117,  0.1829,  ...,  0.2054,  0.0578, -0.0750],
        [ 0.0055, -0.0049, -0.0069,  ..., -0.0030,  0.0038,  0.0087],
        [-0.0448,  0.4604, -0.0604,  ...,  0.1073,  0.0310,  0.0477],
        ...,
        [-0.0138,  0.0278, -0.0467,  ...,  0.0455, -0.0265,  0.0125],
        [-0.0043,  0.0153, -0.0567,  ...,  0.0496,  0.0108, -0.0099],
        [ 0.0053,  0.0324, -0.0179,  ..., -0.0085,  0.0223, -0.0020]],
       requires_grad=True)
Updated the model with ./pretrained_checkpoint/pytorch_model.bin
Parameter containing:
tensor([[-0.0383,  0.1205,  0.1776,  ...,  0.1973,  0.0594, -0.0699],
        [ 0.0046, -0.0023, -0.0084,  ..., -0.0036,  0.0047,  0.0084],
        [-0.0460,  0.4671, -0.0650,  ...,  0.1027,  0.0256,  0.0475],
        ...,
        [-0.0097, -0.0304,  0.0090,  ..., -0.0029,  0.0137, -0.0164],
        [-0.0134, -0.0092, -0.0100,  ..., -0.0006, -0.0203,  0.0194],
        [ 0.0012,  0.0025,  0.0163,  ..., -0.0251, -0.0214, -0.0120

In [19]:
from seq2struct.datasets.spider_lib.preprocess.get_tables import dump_db_json_schema
from seq2struct.datasets.spider import load_tables_from_schema_dict

In [20]:
db_id = "security_1"

In [21]:
my_schema = dump_db_json_schema("data/sqlite_files/{db_id}/{db_id}.sqlite".format(db_id=db_id), db_id)

In [22]:
from seq2struct.utils.api_utils import refine_schema_names

In [23]:
my_schema

{'db_id': 'security_1',
 'table_names_original': ['BASE', 'VARIANT_1'],
 'table_names': ['base', 'variant 1'],
 'column_names_original': [(-1, '*'),
  (0, 'FRAUD_BOOL'),
  (0, 'INCOME'),
  (0, 'NAME_EMAIL_SIMILARITY'),
  (0, 'PREV_ADDRESS_MONTHS_COUNT'),
  (0, 'CURRENT_ADDRESS_MONTHS_COUNT'),
  (0, 'CUSTOMER_AGE'),
  (0, 'DAYS_SINCE_REQUEST'),
  (0, 'INTENDED_BALCON_AMOUNT'),
  (0, 'PAYMENT_TYPE'),
  (0, 'ZIP_COUNT_4W'),
  (0, 'VELOCITY_6H'),
  (0, 'VELOCITY_24H'),
  (0, 'VELOCITY_4W'),
  (0, 'BANK_BRANCH_COUNT_8W'),
  (0, 'DATE_OF_BIRTH_DISTINCT_EMAILS_4W'),
  (0, 'EMPLOYMENT_STATUS'),
  (0, 'CREDIT_RISK_SCORE'),
  (0, 'EMAIL_IS_FREE'),
  (0, 'HOUSING_STATUS'),
  (0, 'PHONE_HOME_VALID'),
  (0, 'PHONE_MOBILE_VALID'),
  (0, 'BANK_MONTHS_COUNT'),
  (0, 'HAS_OTHER_CARDS'),
  (0, 'PROPOSED_CREDIT_LIMIT'),
  (0, 'FOREIGN_REQUEST'),
  (0, 'SOURCE'),
  (0, 'SESSION_LENGTH_IN_MINUTES'),
  (0, 'DEVICE_OS'),
  (0, 'KEEP_ALIVE_SESSION'),
  (0, 'DEVICE_DISTINCT_EMAILS_8W'),
  (0, 'DEVICE_FRAUD_COU

In [34]:
schema, eval_foreign_key_maps = load_tables_from_schema_dict(my_schema)

In [35]:
schema.keys()

dict_keys(['security_1'])

In [36]:
dataset = registry.construct('dataset_infer', {
   "name": "spider", "schemas": schema, "eval_foreign_key_maps": eval_foreign_key_maps, 
    "db_path": "data/sqlite_files/"
})

In [ ]:
We're using the Stanford CoreNLP module to preprocess the schema items

In [37]:
for _, schema in dataset.schemas.items():
    model.preproc.enc_preproc._preprocess_schema(schema)

In [38]:
spider_schema = dataset.schemas[db_id]

In [39]:
def infer(question):
    data_item = SpiderItem(
            text=None,  # intentionally None -- should be ignored when the tokenizer is set correctly
            code=None,
            schema=spider_schema,
            orig_schema=spider_schema.orig,
            orig={"question": question}
        )
    model.preproc.clear_items()
    enc_input = model.preproc.enc_preproc.preprocess_item(data_item, None)
    preproc_data = enc_input, None
    with torch.no_grad():
        output = inferer._infer_one(model, data_item, preproc_data, beam_size=1, use_heuristic=True)
    return output[0]["inferred_code"]

In [40]:
code = infer("What type of device operating systems are used by the customer?")
print(code)

SELECT DISTINCT BASE.DEVICE_OS FROM BASE


In [41]:
code = infer("Can you provide a breakdown of the different payment types used by customers in the database, and how many customers used each payment type in the last month?")
print(code)

SELECT BASE.PAYMENT_TYPE, Count(*) FROM BASE GROUP BY BASE.PAYMENT_TYPE ORDER BY Count(*) Desc LIMIT 1


In [42]:
code = infer("What is the average credit risk score based on age group?")
print(code)

SELECT Avg(BASE.CREDIT_RISK_SCORE), BASE.CUSTOMER_AGE FROM BASE GROUP BY BASE.CUSTOMER_AGE


### Let's see some inferences ran on other database schemas!


In [43]:
db_id = "twitter_1"
my_schema = dump_db_json_schema("data/sqlite_files/{db_id}/{db_id}.sqlite".format(db_id=db_id), db_id)

In [44]:
my_schema

{'db_id': 'twitter_1',
 'table_names_original': ['follows', 'tweets', 'user_profiles'],
 'table_names': ['follows', 'tweets', 'user profiles'],
 'column_names_original': [(-1, '*'),
  (0, 'f1'),
  (0, 'f2'),
  (1, 'id'),
  (1, 'uid'),
  (1, 'text'),
  (1, 'createdate'),
  (2, 'uid'),
  (2, 'name'),
  (2, 'email'),
  (2, 'partitionid'),
  (2, 'followers')],
 'column_names': [(-1, '*'),
  (0, 'f1'),
  (0, 'f2'),
  (1, 'id'),
  (1, 'uid'),
  (1, 'text'),
  (1, 'createdate'),
  (2, 'uid'),
  (2, 'name'),
  (2, 'email'),
  (2, 'partitionid'),
  (2, 'followers')],
 'column_types': ['text',
  'number',
  'number',
  'number',
  'number',
  'text',
  'time',
  'number',
  'text',
  'text',
  'number',
  'number'],
 'primary_keys': [1, 3, 7],
 'foreign_keys': [[2, 7], [1, 7], [4, 7]]}

In [45]:
schema, eval_foreign_key_maps = load_tables_from_schema_dict(my_schema)

In [46]:
dataset = registry.construct('dataset_infer', {
   "name": "spider", "schemas": schema, "eval_foreign_key_maps": eval_foreign_key_maps, 
    "db_path": "data/sqlite_files/"
})

In [47]:
for _, schema in dataset.schemas.items():
    model.preproc.enc_preproc._preprocess_schema(schema)

In [50]:
code = infer("Find the name and email of the user whose name contains the word '%Swift%'")
print(code)

SELECT BASE.NAME_EMAIL_SIMILARITY, BASE.EMAIL_IS_FREE FROM BASE WHERE BASE.NAME_EMAIL_SIMILARITY LIKE 'terminal'
